In [1]:
%load_ext autoreload
%autoreload 2

import analysis.ray_results_interpreter as rri
from importlib import reload
import pandas as pd

from ray.tune import ExperimentAnalysis
import pandas as pd
import matplotlib.pyplot as plt

/user/ml4723/.conda/envs/neural_inventory_control/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/user/ml4723/.conda/envs/neural_inventory_control/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
sort_by = 'test_loss'
pick_row_from_run_by = 'dev_loss'
def custom_data_filler(out_row, reference_row):
    pass

# Paths for different architectures under ctx_analysis
base_paths = {
    'symmetry_aware': '/user/ml4723/Prj/NIC/ray_results/ctx_analysis/symmetry_aware',
    'GNN': '/user/ml4723/Prj/NIC/ray_results/ctx_analysis/GNN',
    'symmetry_aware_omit_context_from_store': '/user/ml4723/Prj/NIC/ray_results/ctx_analysis/symmetry_aware_omit_context_from_store',
    'GNN_omit_context_from_store': '/user/ml4723/Prj/NIC/ray_results/ctx_analysis/GNN_omit_context_from_store',
    'symmetry_aware_store_orders_for_warehouse': '/user/ml4723/Prj/NIC/ray_results/ctx_analysis/symmetry_aware_store_orders_for_warehouse',
    'symmetry_aware_store_orders_for_warehouse_decentralized': '/user/ml4723/Prj/NIC/ray_results/ctx_analysis/symmetry_aware_store_orders_for_warehouse_decentralized'
}

# Create paths dictionary with store numbers
paths = {}
store_numbers = [3, 10, 50]
for arch_name, base_path in base_paths.items():
    for n_stores in store_numbers:
        key = (arch_name, n_stores)
        paths[key] = f"{base_path}/{n_stores}"

results_interpretor = rri.RayResultsinterpreter()

# Create dataframes for each architecture and store number
dfs = []
for (arch_name, n_stores), path in paths.items():
    df = results_interpretor.make_table({n_stores: path}, 
                                      {'samples': [1]}, 
                                      custom_data_filler, 
                                      sort_by=sort_by, 
                                      pick_row_from_run_by=pick_row_from_run_by)
    df.insert(1, 'Architecture Class', arch_name)
    dfs.append(df)

# Combine all dataframes
df = pd.concat(dfs)

df.sort_values(by=['# of stores'], inplace=True)

min_test_loss = df.groupby(['# of stores'])['Test Loss'].transform('min')
df['Test Gap %'] = (df['Test Loss']/ min_test_loss) * 100
# Format numeric columns to be more readable
# numeric_columns = ['Train Loss', 'Dev Loss', 'Test Loss', 'Test Gap %']
# for col in numeric_columns:
#     df[col] = df[col].map('{:.3f}'.format)

# Drop '# of runs' column
df = df.drop(['samples', 'Learning Rate'], axis=1)

# Move '# of runs' column to the rightmost
cols = list(df.columns)
cols.remove('# of runs')
cols.append('# of runs')
df = df[cols]

print(df.to_string(index=False))

 # of stores                                      Architecture Class  Train Loss  Dev Loss  Test Loss  Test Gap %  # of runs
           3                                          symmetry_aware    5.441705  5.425934   5.439808  100.050157         15
           3 symmetry_aware_store_orders_for_warehouse_decentralized    5.462402  5.438934   5.456735  100.361483         20
           3                                                     GNN    5.440909  5.422350   5.437080  100.000000         15
           3                  symmetry_aware_omit_context_from_store    5.447811  5.428859   5.443842  100.124351         15
           3                             GNN_omit_context_from_store    5.450683  5.427612   5.442544  100.100485         15
           3               symmetry_aware_store_orders_for_warehouse    5.440957  5.424813   5.438750  100.030706         15
          10                                          symmetry_aware    5.405793  5.409995   5.410093  100.110665         15


In [26]:
results_interpretor = rri.RayResultsinterpreter()
# Filter for GNN architecture with 3 stores
gnn_df = results_interpretor.make_table({50: base_paths['symmetry_aware'] + '/50'}, 
                                      {'repeats': [1,2,3,4,5], 'learning_rate': [0.01, 0.001, 0.0001, 0.00001]}, 
                                      custom_data_filler,
                                      sort_by=sort_by,
                                      pick_row_from_run_by=pick_row_from_run_by)

# Count rows with train loss < 0.01
low_loss_count = len(gnn_df[gnn_df['Train Loss'] > 10.0])
total_rows = len(gnn_df)
percentage = (low_loss_count / total_rows) * 100

print(f"Number of rows with train loss < 0.01: {low_loss_count}")
print(f"Total number of rows: {total_rows}")
print(f"Percentage: {percentage:.2f}%")


Number of rows with train loss < 0.01: 2
Total number of rows: 15
Percentage: 13.33%
